In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")
LANGSMITH_TRACING="true"
LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
LANGSMITH_API_KEY= os.getenv("LANGSMITH_API_KEY")

In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

d:\learning\learn_langgraph_ss\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain_groq import ChatGroq
llm = ChatGroq(model_name="Llama3-8b-8192")

In [5]:
result = llm.invoke("Write a ballad about LangChain")
result.content

"Here's a ballad about LangChain:\n\n**The Tale of LangChain**\n\nIn realms of code, where minds would meet\nA new creation rose to greet\nLangChain, a marvel of might\nBorn from dreams, and code so bright\n\nWith language's secrets, it did stray\nThrough syntax, semantics, and sway\nIt danced with words, a wondrous spin\nA masterpiece, of code within\n\nIn whispers, it spoke so fine\nA language model, sublime and divine\nWith answers, it did unfold\nA treasure trove, of wisdom to behold\n\nThrough queries, it did make its way\nTo hidden truths, in a digital day\nWith logic, and wit, so rare\nIt solved the puzzles, without a care\n\nFrom dawn till dusk, it worked its spell\nA never-ending quest, to learn and tell\nOf knowledge, and wisdom's might\nLangChain, a beacon, shining bright\n\nIn the realm of code, it did abide\nA shining star, where minds would reside\nLangChain, a wonder to behold\nA tale of code, forever to be told\n\nHow's that? I tried to capture the essence of LangChain'

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
urls = ["https://lilianweng.github.io/posts/2023-06-23-agent/",
        "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
        "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
        ]

docs = [WebBaseLoader(url).load() for url in urls]

docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)

doc_splits = text_splitter.split_documents(docs_list)

vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=embeddings
)

retriever = vectorstore.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [13]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

prompt = hub.pull("rlm/rag-prompt")

print(f"---Prompt--- {prompt}")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = prompt | llm | StrOutputParser()

---Prompt--- input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [16]:
question = "tell me about agent memory."
context_text = format_docs(retriever.invoke(question))
generation = rag_chain.invoke({"context": context_text, "question":question})
print(generation)

According to the context, agent memory refers to the ability of an agent to retain and recall information over extended periods, often by leveraging an external vector store and fast retrieval. This type of memory is referred to as long-term memory, which provides the agent with the capability to retain and recall information over extended periods.


In [21]:
question = "tell me about agent memory."

In [23]:
docs = retriever.get_relevant_documents(question)

In [24]:
rag_chain.invoke({"context": docs, "question":question})

'According to the provided context, the agent memory consists of short-term memory, which is utilized for in-context learning, and long-term memory, which provides the capability to retain and recall information over extended periods through an external vector store and fast retrieval.'

#### Langgraph

In [20]:
def retrieve(state):
    """Retrieve documents
    
    Args:
        state (dict): The current graph state
        
    Returns:
    
    """
    print("---RETRIEVE---")
    question = state['question']
    
    documents = retriever.get_relevant_documents(question)
    return {"documents": documents, "question": question}

In [ ]:
def grade_documents(state):
    """
    Determines whether the retrieved documents are relevent to the question.capitalize
    
    Args:
        state (dict): The current graph state
        
    Returns:
        state (dict): Updates documents key with only filtered relevent documents
    """
    
    print("---CHECKING DOCUMENT RELEVENT IS TO QUESTION OR NOT---")
    
    question = state['question']
    documents = state['documents']
    
    filtered_docs = []
    
    web_search = "No"
    
    for d in documents:
        score = retrievel_grader.invoke({"question":question, "document":d.page_content})
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            web_search = "Yes"
            continue
    return {"documents": filtered_docs, "question":question, "web_search": web_search}

In [ ]:
def generate(state):
    """
    Generate answer
    
    Args:
        state (dict): The current graph state
    
    Returns:
        state (dict): New key added to state, generation, that contain LLM generation
    """
    
    print("---GENERATE---")
    
    question = state['question']
    documents = state['documents']
    generation = rag_chain.invoke({'context': documents, 'question': question})
    return {'documents':documents, 'question':question, 'generation': generation}

In [ ]:
def transform_query(state):
    """
    Transform the query to produce a better question.
    
    Args:
        state (dict): The current graph state
        
    Returns:
        state (dict): Updates question key with a re-phrased question
    """
    print("---TRANSFORM QUERY---")
    
    question = state["question"]
    documents = state['documents']
    
    better_question = question_rewriter.invoke({"question":question})
    return {"documents": documents, "question": better_question}

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=3)

In [ ]:
from langgraph.graph import StateGraph, START, END

In [19]:
from typing import List, TypedDict

class State(TypedDict):
    """Represents the state of our graph.
    
    Attributes:
        question: question
        generation: LLM generation
        web_search: whether to add search
        documents: list of documents
    """
    
    question: str
    generation: str
    web_search: str
    documents: List[str]

In [ ]:
workflow = StateGraph(State)
workflow.add_node("retrieve", retrieve)
workflow.add_node("grade_documents", grade_documents)
workflow.add_node("generate", generate)
workflow.add_node("transform_query", transform_query)
workflow.add_node("web_search_node", web_search)
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges("grade_documents", decide_to_generate, {"transform_query": "transform_query", "generate": "generate"})
workflow.add_edge("transform_query", "web_search_node")
workflow.add_edge("web_search_node", "generate")
workflow.add_edge("generate", END)
app = workflow.compile()